In [584]:
# Chase Abram
# using Pkg
# Pkg.add("DataFrames")
# using CSV
# using DataFrames
# # df = DataFrame()
# df = CSV.read("psetOne.csv", DataFrame)

# Pkg.add("ForwardDiff")
using ForwardDiff
using LinearAlgebra
using Optim

using Random

In [161]:
names(df)
df17 = df[df."Market" .== 17, :]
# df17."shares"
# X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)

7×2 BitArray{2}:
 0  0
 0  0
 1  0
 1  0
 0  1
 0  1
 0  1

In [730]:
function u_bar_jt(delta, sigma, X, zeta)
    return delta .+ X*sigma*zeta
end

function Prij(u_bar)
    return exp.(u_bar)./sum(exp.(u_bar))
end

# Shares
function s_logit(p, alpha, beta, X, xi)
    # Compute numerators for each s_j
#     nums = exp.(X*beta .- alpha.*p[2:end] .+ xi)
    nums = exp.(X*beta .- alpha.*p .+ xi)
    
    # return s_j, starting with s_0
#     return [1/(1 + sum(nums)); nums./(1 + sum(nums))]
    return nums./(1 + sum(nums))
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                # Shares
                s = s_logit(p, alpha, beta, X, xi)
                
                # Jacobian of shares
                sj = s_logit_jac(p, alpha, beta, X, xi)
                
                if j == k
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
#     Om = zeros(size(br,1) + 1,size(br,1) + 1)
    Om = zeros(size(br,1),size(br,1))
    
    for i in 1:size(Om,1)
        for j in 1:size(Om,2)
            if br[i,1] && br[j,1]
                # Brand2
                Om[i,j] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i,j] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i,j] = 1
            end
        end
    end
    
    # Outside option
#     Om[1,1] = 1
    
    return Om
end

# Negative of Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) + Omega(p, alpha, beta, X, xi, br)*p
end

# (Jacobian of Omega)*p (this is used in the gradient of the FP expression)
function jac_om_p(p, alpha, beta, X, xi, br)
    jacomp = zeros(length(p), length(p))
    hess = s_logit_hess(p, alpha, beta, X, xi)
    oms = Omega_star(br)
    for i in 1:length(p)
        for j in 1:length(p)
            jacomp[i,j] = ((hess[i,:,j].* oms[i,j])'* p)[1]
        end
    end
    return jacomp
end

function J_fp_logit(p, alpha, beta, X, xi, br)
    sj = s_logit_jac(p, alpha, beta, X, xi)
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    om = Omega(p, alpha, beta, X, xi, br)
    return sj + jacomp + om
end

function solve_fp_logit(p0, alpha, beta, X, xi, br, tol = 1e-14, maxiter = 100)
    p = p0
    it = 0
    diff = Inf
    while diff > tol && it < maxiter
        J = J_fp_logit(p, alpha, beta, X, xi, br)
        pnew = p - J \ fp_logit(p, alpha, beta, X, xi, br)
        println("pnew: ", pnew)
        diff = maximum(abs.(pnew - p))
        it += 1
        p = pnew
    end
    return p
end



# Squared distance of FP expression from origin
function fp_logit_2(p, alpha, beta, X, xi, br)
    return (fp_logit(p, alpha, beta, X, xi, br)' * fp_logit(p, alpha, beta, X, xi, br))[1]
end


# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     sp = s_logit(p, alpha, beta, X, xi)
#     Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
#     s_jac = s_logit_jac(p, alpha, beta, X, xi)
#     jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
#     jac = 2*(sp + Om*p)' * (s_jac + jacomp + Om)
#     for i in 1:length(p)
#         storage[i] = jac[i]
#     end
# #     storage = (sp + Om*p)' * (s_jac + jacomp + Om)
# #     storage = 0
# #     return storage
    
# end

# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     s = s_logit(p, alpha, beta, X, xi)
#     oms = Omega_star(br)
#     sj = s_logit_jac(p, alpha, beta, X, xi)
#     sh = s_logit_hess(p, alpha, beta, X, xi)
    
#     for k in 1:length(p)
#         storage[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
#     end
#     println("stor: ", storage)
# #     return storage
# end

function fp_logit_2_jac(p, alpha, beta, X, xi, br)
    s = s_logit(p, alpha, beta, X, xi)
    oms = Omega_star(br)
    sj = s_logit_jac(p, alpha, beta, X, xi)
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    out = zeros(length(p))
    for k in 1:length(p)
        out[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
    end
#     println("stor: ", storage)
    return out
end

function basic_newton_zero(f, g, init, tol = 1e-14, maxiter = 10000)
    x = init
    x_new = init
    diff = Inf
    it = 1
#     println("x init: ", init)
    while diff > tol && it < maxiter
#         println("it: ", it)
#         println("g(x): ", g(x))
        x_new -= 0.5.*g(x)
#         println("x_new: ", x_new)
        diff = maximum(abs.(x_new - x))
#         println("diff: ", diff)
        it += 1
        x = x_new
    end
    return x
end


# function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    
#     f(x) = fp_logit_2(x, alpha, beta, X, xi, br)[1,1]
    
# #     g!(storage, x) = fp_logit_2_jac(storage, x, alpha, beta, X, xi, br)
#     g!(x) = fp_logit_2_jac(x, alpha, beta, X, xi, br)
    
#     out = basic_newton_zero(f, g!, p)
# #     Optim.Options(f_tol = tol)
# #     Optim.Options(time_limit = 10.0)
# #     opt = optimize(f, g!, p, LBFGS(), Optim.Options(time_limit = 5.0))
# #     opt = optimize(f, x -> ForwardDiff.gradient(f, p), p, BFGS(), Optim.Options(time_limit = 10.0))
# #     return opt, opt.minimizer, opt.minimum
#     return out
# end



basic_newton_zero (generic function with 3 methods)

In [731]:
p = ones(size(X,1),1)
p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'

# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1))

s_logit(p, alpha, beta, X, xi)
# println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)

s_logit_hess(p, alpha, beta, X, xi)

Omega_star(Br)

Omega(p, alpha, beta, X, xi, Br)

fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)

solve_fp_logit(p, alpha, beta, X, xi, Br)

# inv(J_fp_logit(p, alpha, beta, X, xi, Br))

# fp_logit_2(p, alpha, beta, X, xi, Br)
# st = zeros(length(p))
# fp_logit_2_jac(st, p, alpha, beta, X, xi, Br)
# println(st)

# jac_om_p(p, alpha, beta, X, xi, Br)

# fp_logit_jac(p, alpha, beta, X, xi, Br)


# op = solve_fp_logit(alpha, beta, X, xi, Br)
# println("op: ", op)
# println("opam: ", opam)
# println("test on fp: ", fp_logit(opam, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))
# df17[3, "Brand2"]

# Gradient must be wrong.
# Check whole construction and parameter choice.

pnew: [452.10747615171044; 452.1074761517107; 452.0746914398183; 452.0746914398183; 903.9021998312282; 903.9021998312279; 903.902199831228]


LoadError: SingularException(1)

In [430]:
# NM: opam: [0.3720742469890362; 0.3724406061806239; 0.3447723065789802; 
# 0.3424326539927414; 2.4893064284464703; 2.5105275013368717; 4.366510826679992]

# BFGS: opam: [0.37346117252339095; 0.37346116923202516; 5.8690207005673365; 
# 0.34160603728869476; 2.5287477985474984; 2.5287470025674392; 2.528747016448347]







sHat (generic function with 1 method)

In [707]:
function f1(x, y, z) 
    return (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
end

function g1(storage, x, y, z)
    storage[1] = 2*(x[1] - 5 - y)
    storage[2] = 2*(x[2] - 3 - z)
end

function g2(x,y,z)
    out = zeros(2)
    out[1] = 2*(x[1] - 5 - y)
    out[2] = 2*(x[2] - 3 - z)
    
    return out
end

function tester()
    a = 1000
    b = -1000
    f(x) = f1(x,a,b)
#     g!(storage, x) = g1(storage, x, a, b)
    g!(x) = g2(x,a,b)
    
    out = basic_newton_zero(f, g!, [-200.0, 1000.0])
    
#     opt = optimize(f, g!, [100.0, -20.0], LBFGS())
#     return opt, opt.minimizer, opt.minimum
    return out
end

tester()



x init: [-200.0, 1000.0]
it: 1
g(x): [-2410.0, 3994.0]
x_new: [1005.0, -997.0]
diff: 1997.0
it: 2
g(x): [0.0, 0.0]
x_new: [1005.0, -997.0]
diff: 0.0


2-element Array{Float64,1}:
 1005.0
 -997.0

In [734]:
p = ones(size(X,1),1)
# p[1] = 50
# p[2] = 30
p[3] = 90
# p[4] = -12
# p[5] = -40
# p[6] = 0
# p[7] = 1
s(x) = s_logit(x, alpha, beta, X, xi)
println(s_logit_jac(p, alpha, beta, X, xi))


# ForwardDiff.gradient(fp, p)
ForwardDiff.jacobian(s, p)

# fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
# println(fp_logit_2_jac(p, alpha, beta, X, xi, Br))
# ForwardDiff.gradient(fp, p)

fp(x) = fp_logit(x, alpha, beta, X, xi, Br)
println(J_fp_logit(p, alpha, beta, X, xi, Br))
ForwardDiff.jacobian(fp, p)


[-0.00025552111259409316 2.742134873982967e-8 2.267000769053989e-125 1.0075949868027336e-8 2.6440102646489126e-7 0.00024893402401267854 6.28473501807021e-6; 2.742134873982967e-8 -0.00032188421285230445 2.8558419204298085e-125 1.2693123184633722e-8 3.330777587244502e-7 0.0003135932862931276 7.917160844584898e-6; 2.2670007690539891e-125 2.8558419204298085e-125 -2.6613937776872866e-121 1.0493765384875231e-125 2.7536484158648933e-124 2.5925647492461886e-121 6.545341694782582e-123; 1.0075949868027338e-8 1.2693123184633722e-8 1.0493765384875231e-125 -0.00011828409280914072 1.2238912209988222e-7 0.00011522957027456282 2.909153613267599e-6; 2.6440102646489126e-7 3.330777587244502e-7 2.7536484158648937e-124 1.2238912209988222e-7 -0.00310078050601391 0.0030237165784616498 7.633853002114934e-5; 0.0002489340240126786 0.0003135932862931276 2.5925647492461886e-121 0.00011522957027456282 0.00302371657846165 -0.0755795377426165 0.07187285813317648; 6.28473501807021e-6 7.917160844584898e-6 6.5453416947

7×7 Array{Float64,2}:
  0.000255308   -1.09654e-7    -4.53138e-125  …  -0.000497581   -1.25622e-5
 -1.09654e-7     0.000321595   -5.70839e-125     -0.000626824   -1.58252e-5
 -6.09823e-123  -7.68221e-123   7.13253e-119     -6.97399e-119  -1.7607e-120
 -2.01495e-8    -2.53832e-8    -2.84381e-123     -0.000230432   -5.81762e-6
  1.05723e-6     1.33184e-6     1.10107e-123      0.00604319     0.00015257
  0.000995387    0.00125393     1.03666e-120  …  -0.153102       0.143645
  2.51301e-5     3.16575e-5     2.61722e-122      0.143645      -0.143884

In [ ]:
function sHat(delta, X, sigma, zeta)
    
    
    # Compute u_bar_ijt
    
    # Compute prob.
    
    # Do above I times, then integrate
    
    
    return 0
end